In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mcfly
from sklearn.model_selection import train_test_split
import gc
import os
from mcfly.find_architecture import train_models_on_samples
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from imblearn.over_sampling import SMOTE
from keras import backend as K
import keras
import json
import csv

split_path = "400_dumped/Final_Data/split/train_val_test.json"

# conversion table btw datasets

In [ ]:
brs_scores = pd.read_csv("GeneticData/BRS_PRS_Scores.txt", sep="\t")
brs_scores['IID'] = brs_scores['IID'].map(lambda a: a.split('_', 1)[1])
brs_scores = brs_scores.set_index('IID')
print(brs_scores.shape[0])

In [ ]:
ids_translation_table = pd.read_csv("GeneticData/merged-dataset-20181231.txt", sep="\t")
ids_translation_table = ids_translation_table.set_index('IID')
print(ids_translation_table.shape[0])

In [ ]:
brs_translation_table = pd.concat([ids_translation_table[['ID']], brs_scores[['SCORE']]], axis=1)
print(brs_scores.shape[0], ids_translation_table.shape[0], brs_translation_table.shape[0], brs_translation_table.dropna().shape[0])
brs_translation_table

In [ ]:
ecg_neg_translation_table = pd.read_csv("BrS_negatives_translation_table.csv", sep=",")
ecg_pos_translation_table = pd.read_csv("BrS_positives_translation_table.csv", sep=",")
print(ecg_neg_translation_table.shape[0], ecg_pos_translation_table.shape[0])

ecg_translation_table = pd.concat([ecg_neg_translation_table, ecg_pos_translation_table])
ecg_translation_table = ecg_translation_table.rename(columns={"original_patient_id": "ID"})

ecg_translation_table = ecg_translation_table.set_index('ID')
ids_translation_table = ids_translation_table.reset_index()
ids_translation_table = ids_translation_table.set_index('ID')

In [ ]:
ids_translation_table.columns = ids_translation_table.columns.str.replace(' ', '')
ecg_translation_table.columns = ecg_translation_table.columns.str.replace(' ', '')
brs_translation_table.columns = brs_translation_table.columns.str.replace(' ', '')

In [ ]:
ecg_transl_table = pd.concat([ecg_translation_table, ids_translation_table['IID']], axis=1)
print(ecg_translation_table.shape[0], ids_translation_table.shape[0], ecg_transl_table.shape[0], ecg_transl_table.dropna().shape[0])
ecg_transl_table = ecg_transl_table.dropna()
ecg_transl_table
                                                                        

In [ ]:
brs_translation_table = brs_translation_table.dropna()
brs_translation_table = brs_translation_table.reset_index()
brs_translation_table = brs_translation_table.set_index('ID')
brs_translation_table

In [ ]:
genetic_data_available = pd.concat([ecg_transl_table, brs_translation_table], axis=1)
genetic_data_available = genetic_data_available.dropna()
genetic_data_available = genetic_data_available.reset_index()
genetic_data_available = genetic_data_available.T.drop_duplicates().T
genetic_data_available = genetic_data_available[['anonymous_id', 'SCORE']]
genetic_data_available['anonymous_id'] = genetic_data_available['anonymous_id'].astype(int)

In [ ]:
with open("GeneticData/available_BRS_PRS.txt", 'w') as f:
# create the csv writer
    writer = csv.writer(f)

    header = np.array(genetic_data_available.columns)
    writer.writerow(header)
    for row in range(genetic_data_available.shape[0]):
        # write a row to the csv file
        writer.writerow(np.array(genetic_data_available.iloc[row]))